In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import gc
import sys
import json
import warnings

import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import RobustScaler

sys.path.append(str(Path().resolve().parent))

from src.utils import (
    seed_every_thing,
    fetch_data,
    Config,
    plot_metric,
    reduce_tf_gpu_memory,
    reduce_mem_usage,
    fetch_custom_data,
    CustomL1Loss
)
warnings.simplefilter('ignore')
pd.options.display.max_columns = 200

In [2]:
basedir = Path().resolve().parent
logdir = basedir / "logs"
datadir = basedir / "data"
cachedir = basedir / "cache"

In [3]:
rc_preds = pd.read_csv(logdir / "cnn-classify-rc-reliable_3" / "valid_preds.csv")
rc_test_preds = pd.read_csv(logdir / "cnn-classify-rc-reliable_3" / "test_preds.csv")

In [4]:
train = pd.read_csv(cachedir / "train-10fold-debugFalse.csv")
train.head()

,time_step,u_in,u_out,pressure,kfold,corss,cross2,cross3,time_delta,time_step_cumsum,u_in_cumsum,count,u_in_cummean,u_in_lag_b1,u_out_lag_b1,u_in_lag_b2,u_out_lag_b2,u_in_lag_b3,u_out_lag_b3,u_in_lag_b4,u_out_lag_b4,u_in_lag_b5,u_out_lag_b5,u_in_lag_f1,u_out_lag_f1,u_in_lag_f2,u_out_lag_f2,u_in_lag_f3,u_out_lag_f3,u_in_lag_f4,u_out_lag_f4,u_in_lag_f5,u_out_lag_f5,u_in_diff_b1,u_out_diff_b1,u_in_diff_b2,u_out_diff_b2,u_in_diff_b3,u_out_diff_b3,u_in_diff_b4,u_out_diff_b4,u_in_diff_b5,u_out_diff_b5,u_in_diff_f1,u_out_diff_f1,u_in_diff_f2,u_out_diff_f2,u_in_diff_f3,u_out_diff_f3,u_in_diff_f4,u_out_diff_f4,u_in_diff_f5,u_out_diff_f5,u_in_bwindow_mean,u_in_bwindow_max,u_in_bwindow_min,u_in_bwindow_std,u_in_fwindow_mean,u_in_fwindow_max,u_in_fwindow_min,u_in_fwindow_std,u_in_cwindow_mean,u_in_cwindow_max,u_in_cwindow_min,u_in_cwindow_std,u_in_bwindow_ewm,u_in_fwindow_ewm,u_in_cwindow_ewm,u_in_bwindow_mean_diff,u_in_bwindow_max_diff,u_in_bwindow_min_diff,u_in_fwindow_mean_diff,u_in_fwindow_max_diff,u_in_fwindow_min_diff,u_in_cwindow_mean_diff,u_in_cwindow_max_diff,u_in_cwindow_min_diff,area,area_insp,area_insp_last,u_in_max,u_in_mean,u_in_std,area_max,area_insp_max,area_mean,area_insp_mean,vibs,RC_20_10,RC_20_20,RC_20_50,RC_50_10,RC_50_20,RC_50_50,RC_5_10,RC_5_20,RC_5_50,R_20,R_5,R_50,C_10,C_20,C_50,norm_time_step,u_in_max_diff,area_max_diff,area_insp_max_diff,u_in_mean_diff,area_mean_diff,area_insp_mean_diff
0,0.00000,0.0833,0,5.836,4,0.0,0.00000,0.0,0.00000,0.00000,0.0833,1,0.0833,0.0000,0.0,0.0000,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,18.39,0.0,22.52,0.0,22.81,0.0,25.36,0.0,27.27,0.0,0.0000,0.0,0.000,0.0,0.000,0.0,0.00,0.0,0.0,0.0,-18.3000,0.0,-22.420,0.0,-22.720,0.0,-25.270,0.0,-27.170,0.0,0.013885,0.0833,0.0,0.03105,19.40,27.27,0.0833,9.0600,5.120,22.52,0.0,8.91,0.05957,26.45,20.69,0.06946,0.0,0.0833,-19.310,-27.170,0.0,-5.040,-22.420,0.0833,0.0000,0.0000,22.6,28.31,10.15,10.125,27.64,22.6,19.66,18.64,14,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0.00000,-28.230,-27.64,-22.60,-10.060,-19.66,-18.640
1,0.03366,18.3900,0,5.906,4,0.0,0.03366,0.0,0.03366,0.03366,18.4700,2,9.2340,0.0833,0.0,0.0000,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,22.52,0.0,22.81,0.0,25.36,0.0,27.27,0.0,27.12,0.0,18.3000,0.0,0.000,0.0,0.000,0.0,0.00,0.0,0.0,0.0,-4.1250,0.0,-4.426,0.0,-6.973,0.0,-8.875,0.0,-8.740,0.0,3.078000,18.3900,0.0,6.84400,23.90,27.27,18.3900,3.0920,7.973,22.81,0.0,10.34,13.16000,26.94,22.34,15.30500,0.0,18.3900,-5.523,-8.875,0.0,10.410,-4.426,18.3900,0.6187,0.6187,22.6,28.31,10.15,10.125,27.64,22.6,19.66,18.64,14,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0.01251,-9.930,-27.02,-21.97,8.234,-19.03,-18.030
2,0.06750,22.5200,0,7.875,4,0.0,0.06750,0.0,0.03387,0.10120,40.9700,3,13.6600,18.3900,0.0,0.0833,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,22.81,0.0,25.36,0.0,27.27,0.0,27.12,0.0,26.81,0.0,4.1250,0.0,22.420,0.0,0.000,0.0,0.00,0.0,0.0,0.0,-0.2996,0.0,-2.846,0.0,-4.750,0.0,-4.617,0.0,-4.297,0.0,6.830000,22.5200,0.0,9.70000,25.31,27.27,22.5200,1.9770,11.140,25.36,0.0,11.26,19.84000,26.84,24.69,15.68000,0.0,22.5200,-2.803,-4.750,0.0,11.370,-2.846,22.5200,1.3810,1.3810,22.6,28.31,10.15,10.125,27.64,22.6,19.66,18.64,14,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0.02510,-5.805,-26.25,-21.22,12.360,-18.27,-17.270
3,0.10156,22.8100,0,11.740,4,0.0,0.10156,0.0,0.03403,0.20280,63.7800,4,15.9450,22.5200,0.0,18.3900,0.0,0.0833,0.0,0.0000,0.0,0.0,0.0,25.36,0.0,27.27,0.0,27.12,0.0,26.81,0.0,27.86,0.0,0.2996,0.0,4.426,0.0,22.720,0.0,0.00,0.0,0.0,0.0,-2.5470,0.0,-4.450,0.0,-4.320,0.0,-3.998,0.0,-5.055,0.0,10.630000,22.8100,0.0,10.69500,26.20,27.86,22.8100,1.7000,14.550,27.27,0.0,11.49,21.97000,27.58,26.69,12.18000,0.0,22.8100,-3.395,-5.055,0.0,8.260,-4.450,22.8100,2.1560,2.1560,22.6,28.31,10.15,10.125,27.64,22.6,19.66,18.64,14,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0.03775,-5.504,-25.48,-20.44,12.664,-17.48,-16.480
4,0.13570,25.3600,0,12.234,4,0.0,0.13570,0.0,0.03420,0.33840,89.1000,5,17.8300,22.8100,0.0,22.5200,0.0,18.3900,0.0,0.0833,0.0,0.0,0.0,27.27,0.0,27.12,0.0,26.81,0.0,27.86,0.0,28.31,0.0,2.5470,0.0,2.846,0.0,6.973,

In [7]:
test = pd.read_csv(cachedir / "test-10fold-debugFalse.csv")
test.head()

,time_step,u_in,u_out,corss,cross2,cross3,time_delta,time_step_cumsum,u_in_cumsum,count,u_in_cummean,u_in_lag_b1,u_out_lag_b1,u_in_lag_b2,u_out_lag_b2,u_in_lag_b3,u_out_lag_b3,u_in_lag_b4,u_out_lag_b4,u_in_lag_b5,u_out_lag_b5,u_in_lag_f1,u_out_lag_f1,u_in_lag_f2,u_out_lag_f2,u_in_lag_f3,u_out_lag_f3,u_in_lag_f4,u_out_lag_f4,u_in_lag_f5,u_out_lag_f5,u_in_diff_b1,u_out_diff_b1,u_in_diff_b2,u_out_diff_b2,u_in_diff_b3,u_out_diff_b3,u_in_diff_b4,u_out_diff_b4,u_in_diff_b5,u_out_diff_b5,u_in_diff_f1,u_out_diff_f1,u_in_diff_f2,u_out_diff_f2,u_in_diff_f3,u_out_diff_f3,u_in_diff_f4,u_out_diff_f4,u_in_diff_f5,u_out_diff_f5,u_in_bwindow_mean,u_in_bwindow_max,u_in_bwindow_min,u_in_bwindow_std,u_in_fwindow_mean,u_in_fwindow_max,u_in_fwindow_min,u_in_fwindow_std,u_in_cwindow_mean,u_in_cwindow_max,u_in_cwindow_min,u_in_cwindow_std,u_in_bwindow_ewm,u_in_fwindow_ewm,u_in_cwindow_ewm,u_in_bwindow_mean_diff,u_in_bwindow_max_diff,u_in_bwindow_min_diff,u_in_fwindow_mean_diff,u_in_fwindow_max_diff,u_in_fwindow_min_diff,u_in_cwindow_mean_diff,u_in_cwindow_max_diff,u_in_cwindow_min_diff,area,area_insp,area_insp_last,u_in_max,u_in_mean,u_in_std,area_max,area_insp_max,area_mean,area_insp_mean,vibs,RC_20_10,RC_20_20,RC_20_50,RC_50_10,RC_50_20,RC_50_50,RC_5_10,RC_5_20,RC_5_50,R_20,R_5,R_50,C_10,C_20,C_50,norm_time_step,u_in_max_diff,area_max_diff,area_insp_max_diff,u_in_mean_diff,area_mean_diff,area_insp_mean_diff
0,0.00000,0.000,0,0.0,0.00000,0.0,0.00000,0.0000,0.000,1,0.000,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,7.516,0.0,14.65,0.0,21.23,0.0,26.33,0.0,30.48,0.0,0.000,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,-7.516,0.0,-14.650,0.0,-21.230,0.0,-26.330,0.0,-30.48,0.0,0.000,0.000,0.0,0.000,16.70,30.48,0.000,10.570,2.771,14.65,0.0,5.120,0.00,28.69,12.695,0.00,0.0,0.000,-16.700,-30.48,0.0,-2.771,-14.650,0.000,0.0000,0.0000,19.48,37.53,9.33,12.18,23.81,19.48,17.48,16.69,4,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0.000000,-37.53,-23.81,-19.48,-9.3300,-17.480,-16.690
1,0.03190,7.516,0,0.0,0.03190,0.0,0.03190,0.0319,7.516,2,3.758,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,14.650,0.0,21.23,0.0,26.33,0.0,30.48,0.0,33.53,0.0,7.516,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,-7.137,0.0,-13.720,0.0,-18.810,0.0,-22.970,0.0,-26.03,0.0,1.253,7.516,0.0,2.800,22.30,33.53,7.516,9.016,5.426,21.23,0.0,7.797,5.37,32.16,19.330,6.26,0.0,7.516,-14.770,-26.03,0.0,2.090,-13.720,7.516,0.2397,0.2397,19.48,37.53,9.33,12.18,23.81,19.48,17.48,16.69,4,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0.012665,-30.03,-23.58,-19.25,-1.8125,-17.250,-16.450
2,0.06384,14.650,0,0.0,0.06384,0.0,0.03192,0.0957,22.170,3,7.390,7.516,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,21.230,0.0,26.33,0.0,30.48,0.0,33.53,0.0,35.72,0.0,7.137,0.0,14.65,0.0,0.00,0.0,0.00,0.0,0.0,0.0,-6.580,0.0,-11.670,0.0,-15.836,0.0,-18.890,0.0,-21.06,0.0,3.695,14.650,0.0,5.617,27.00,35.72,14.650,7.270,8.710,26.33,0.0,10.050,12.01,34.72,24.770,10.96,0.0,14.650,-12.340,-21.06,0.0,5.938,-11.670,14.650,0.7075,0.7075,19.48,37.53,9.33,12.18,23.81,19.48,17.48,16.69,4,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0.025330,-22.89,-23.11,-18.78,5.3240,-16.780,-15.984
3,0.09576,21.230,0,0.0,0.09576,0.0,0.03192,0.1915,43.400,4,10.850,14.650,0.0,7.516,0.0,0.000,0.0,0.0,0.0,0.0,0.0,26.330,0.0,30.48,0.0,33.53,0.0,35.72,0.0,36.97,0.0,6.580,0.0,13.72,0.0,21.23,0.0,0.00,0.0,0.0,0.0,-5.090,0.0,-9.260,0.0,-12.310,0.0,-14.484,0.0,-15.74,0.0,7.234,21.230,0.0,8.250,30.72,36.97,21.230,5.500,12.520,30.48,0.0,11.664,18.61,36.34,29.220,14.00,0.0,21.230,-9.484,-15.74,0.0,8.700,-9.260,21.230,1.3850,1.3850,19.48,37.53,9.33,12.18,23.81,19.48,17.48,16.69,4,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0.038000,-16.31,-22.42,-18.11,11.9100,-16.110,-15.305
4,0.12770,26.330,0,0.0,0.12770,0.0,0.03190,0.3190,69.700,5,13.945,21.230,0.0,14.650,0.0,7.516,0.0,0.0,0.0,0.0,0.0,30.480,0.0,33.53,0.0,35.72,0.0,36.97,0.0,37.53,0.0,5.090,0.0,11.67,0.0,18.81,0.0,26.33,0.0,0.0,0.0,-4.164,0.0,-7.227,0.0,-9.400,0.0,-10.650,0.0,-11.22,0.0,11.620,26.330,0.0,10.040,33.44,37.53,26.330,3.955,16.720,33.53,0.0,12.414,24.12,37.22,32.600,14.70,0

In [18]:
train = train[train["count"] <= 35].reset_index(drop=True)
test = test[test["count"] <= 35].reset_index(drop=True)

In [13]:
rc_cols = [c for c in train.columns if "RC_" in c]
r_cols = ["R_20", "R_5", "R_50"]
c_cols = ["C_10", "C_20", "C_50"]

In [28]:
train[rc_cols] += np.tile(rc_preds, (1, 35)).reshape(-1, rc_preds.shape[1])
test[rc_cols] += np.tile(rc_test_preds, (1, 35)).reshape(-1, rc_test_preds.shape[1])

In [30]:
train.to_csv(cachedir / "train-10fold-cut-soft-rc-debugFalse.csv", index=False)
train.iloc[:1000 * 35].to_csv(cachedir / "train-10fold-cut-soft-rc-debugTrue.csv", index=False)

In [31]:
test.to_csv(cachedir / "test-10fold-cut-soft-rc-debugFalse.csv", index=False)
test.iloc[:1000 * 35].to_csv(cachedir / "test-10fold-cut-soft-rc-debugTrue.csv", index=False)

In [36]:
train.drop(r_cols + c_cols, axis=1).to_csv(cachedir / "train-10fold-cut-only-soft-rc-debugFalse.csv", index=False)
train.drop(r_cols + c_cols, axis=1).iloc[:1000 * 35].to_csv(cachedir / "train-10fold-cut-only-soft-rc-debugTrue.csv", index=False)

In [37]:
test.drop(r_cols + c_cols, axis=1).to_csv(cachedir / "test-10fold-cut-only-soft-rc-debugFalse.csv", index=False)
test.drop(r_cols + c_cols, axis=1).iloc[:1000 * 35].to_csv(cachedir / "test-10fold-cut-only-soft-rc-debugTrue.csv", index=False)